In [44]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import morphology, filters, measure
from skimage.morphology import disk, area_opening
from scipy.ndimage import median_filter
from sklearn.metrics import jaccard_score
from skimage.measure import regionprops
import skimage
from scans import *

In [45]:
def plot_slice(slice, title):
    plt.imshow(slice, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
    
def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

# Morphological Operations
def morphological_operations(image):
    selem = disk(2)
    eroded = morphology.erosion(image, selem)
    return eroded

# Image Filtering
def image_filtering(image):
    return median_filter(image, size=5)

# Threshold Segmentation
def threshold_segmentation(image):
    threshold = filters.threshold_otsu(image)
    binary = image > threshold
    return binary

# Small Object Removal
def small_object_removal(image, min_size=100):
    return morphology.remove_small_objects(image, min_size)

# Modify Small Object Removal
def selective_object_removal(image):
    labeled_image, num_labels = skimage.measure.label(image, connectivity=2, return_num=True)
    
    # Find the largest connected component
    largest_region = None
    max_region_area = 0
    for region in regionprops(labeled_image):
        if region.area > max_region_area:
            max_region_area = region.area
            largest_region = region
            
    # Create a new image where only the largest connected component is retained
    cleaned_image = np.zeros_like(image)
    if largest_region:
        minr, minc, maxr, maxc = largest_region.bbox
        cleaned_image[minr:maxr, minc:maxc] = largest_region.image
    
    return cleaned_image

def image_subtraction(image1, image2):
    return np.bitwise_xor(image1, image2).astype(np.uint8)

In [46]:
# File paths
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [47]:
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[:21]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']

        dscs = []
        sensitivities = []
        specificities = []
        accuracies = []
        weights = []

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)

            morph_image = morphological_operations(slice)
            filtered_image = image_filtering(morph_image)
            thresh_image = threshold_segmentation(filtered_image)
            cleaned_image = selective_object_removal(thresh_image)
            segmented_mask = image_subtraction(cleaned_image, thresh_image)

            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))

            # Calculate DSC, Sensitivity, Specificity, Accuracy
            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            dscs.append(dsc)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            accuracies.append(accuracy)
            weights.append(np.sum(segmented_mask))

        weighted_mean_dsc = np.average(dscs, weights=weights)
        weighted_std_dsc = np.sqrt(np.average((dscs - weighted_mean_dsc) ** 2, weights=weights))
        mean_dsc = np.mean(dscs)  # Calculate mean DSC
        weighted_mean_sensitivity = np.average(sensitivities, weights=weights)
        weighted_std_sensitivity = np.sqrt(np.average((sensitivities - weighted_mean_sensitivity) ** 2, weights=weights))
        mean_sensitivity = np.mean(sensitivities)
        weighted_mean_specificity = np.average(specificities, weights=weights)
        weighted_std_specificity = np.sqrt(np.average((specificities - weighted_mean_specificity) ** 2, weights=weights))
        mean_specificity = np.mean(specificities)
        weighted_mean_accuracy = np.average(accuracies, weights=weights)
        weighted_std_accuracy = np.sqrt(np.average((accuracies - weighted_mean_accuracy) ** 2, weights=weights))
        mean_accuracy = np.mean(accuracies)
        
        results_file.write(f'Scan {scan_number}:\n'
                           f'  - Weighted Mean DSC: {weighted_mean_dsc}, '
                           f'Standard Deviation: {weighted_std_dsc}, Mean DSC: {mean_dsc}\n'
                           f'  - Weighted Mean Sensitivity: {weighted_mean_sensitivity}, '
                           f'Standard Deviation: {weighted_std_sensitivity}, Mean Sensitivity: {mean_sensitivity}\n'
                           f'  - Weighted Mean Specificity: {weighted_mean_specificity}, '
                           f'Standard Deviation: {weighted_std_specificity}, Mean Specificity: {mean_specificity}\n'
                           f'  - Weighted Mean Accuracy: {weighted_mean_accuracy}, '
                           f'Standard Deviation: {weighted_std_accuracy}, Mean Accuracy: {mean_accuracy}\n\n')

print("Results saved in 'results.txt'")

Scan 001 - Slice 231: 0.0
Scan 001 - Slice 232: 0.1864406779661017
Scan 001 - Slice 233: 0.3008130081300813
Scan 001 - Slice 234: 0.5482866043613707
Scan 001 - Slice 235: 0.6827956989247311
Scan 001 - Slice 236: 0.6464646464646465
Scan 001 - Slice 237: 0.6896551724137931
Scan 001 - Slice 238: 0.7078891257995735
Scan 001 - Slice 239: 0.6727688787185355
Scan 001 - Slice 240: 0.6871508379888268
Scan 001 - Slice 241: 0.5981873111782477
Scan 001 - Slice 242: 0.5746268656716418
Scan 001 - Slice 243: 0.4368932038834951
Scan 001 - Slice 244: 0.39823008849557523
Scan 001 - Slice 245: 0.3926940639269406
Scan 001 - Slice 246: 0.32044198895027626
Scan 001 - Slice 247: 0.07272727272727272
Scan 001 - Slice 248: 0.0
Scan 001 - Slice 250: 0.0
Scan 003 - Slice 138: 0.0912863070539419
Scan 003 - Slice 139: 0.29253731343283584
Scan 003 - Slice 140: 0.3972286374133949
Scan 003 - Slice 141: 0.4279661016949153
Scan 003 - Slice 142: 0.3391003460207612
Scan 003 - Slice 143: 0.30718954248366015
Scan 003 - Slic